Lambda School Data Science

*Unit 1, Sprint 1, Module 3*

---

# Join and Reshape datasets

Objectives
- concatenate data with pandas
- merge data with pandas
-  understand tidy data formatting
-  melt and pivot data with pandas

Links
- [Pandas Cheat Sheet](https://github.com/pandas-dev/pandas/blob/master/doc/cheatsheet/Pandas_Cheat_Sheet.pdf)
- [Tidy Data](https://en.wikipedia.org/wiki/Tidy_data)
  - Combine Data Sets: Standard Joins
  - Tidy Data
  - Reshaping Data
- Python Data Science Handbook
  - [Chapter 3.6](https://jakevdp.github.io/PythonDataScienceHandbook/03.06-concat-and-append.html), Combining Datasets: Concat and Append
  - [Chapter 3.7](https://jakevdp.github.io/PythonDataScienceHandbook/03.07-merge-and-join.html), Combining Datasets: Merge and Join
  - [Chapter 3.8](https://jakevdp.github.io/PythonDataScienceHandbook/03.08-aggregation-and-grouping.html), Aggregation and Grouping
  - [Chapter 3.9](https://jakevdp.github.io/PythonDataScienceHandbook/03.09-pivot-tables.html), Pivot Tables
  
Reference
- Pandas Documentation: [Reshaping and Pivot Tables](https://pandas.pydata.org/pandas-docs/stable/reshaping.html)
- Modern Pandas, Part 5: [Tidy Data](https://tomaugspurger.github.io/modern-5-tidy.html)

In [7]:
!wget https://s3.amazonaws.com/instacart-datasets/instacart_online_grocery_shopping_2017_05_01.tar.gz

--2020-01-23 02:52:48--  https://s3.amazonaws.com/instacart-datasets/instacart_online_grocery_shopping_2017_05_01.tar.gz
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.217.41.70
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.217.41.70|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 205548478 (196M) [application/x-gzip]
Saving to: ‘instacart_online_grocery_shopping_2017_05_01.tar.gz’

instacart_online_gr 100%[===================>] 196.03M  16.6MB/s    in 13s     

2020-01-23 02:53:02 (14.7 MB/s) - ‘instacart_online_grocery_shopping_2017_05_01.tar.gz’ saved [205548478/205548478]



In [2]:
!tar --gunzip --extract --verbose --file=instacart_online_grocery_shopping_2017_05_01.tar.gz

instacart_2017_05_01/
instacart_2017_05_01/._aisles.csv
instacart_2017_05_01/aisles.csv
instacart_2017_05_01/._departments.csv
instacart_2017_05_01/departments.csv
instacart_2017_05_01/._order_products__prior.csv
instacart_2017_05_01/order_products__prior.csv
instacart_2017_05_01/._order_products__train.csv
instacart_2017_05_01/order_products__train.csv
instacart_2017_05_01/._orders.csv
instacart_2017_05_01/orders.csv

gzip: stdin: unexpected end of file
tar: Unexpected EOF in archive
tar: Unexpected EOF in archive
tar: Error is not recoverable: exiting now


In [3]:
%cd instacart_2017_05_01

/content/instacart_2017_05_01


In [4]:
!ls -lh *.csv

-rw-r--r-- 1  502 staff 2.6K May  2  2017 aisles.csv
-rw-r--r-- 1  502 staff  270 May  2  2017 departments.csv
-rw-r--r-- 1  502 staff 551M May  2  2017 order_products__prior.csv
-rw-r--r-- 1  502 staff  24M May  2  2017 order_products__train.csv
-rw------- 1 root root   77M Jan 23 02:42 orders.csv


# Assignment

## Join Data Practice

These are the top 10 most frequently ordered products. How many times was each ordered? 

1. Banana
2. Bag of Organic Bananas
3. Organic Strawberries
4. Organic Baby Spinach 
5. Organic Hass Avocado
6. Organic Avocado
7. Large Lemon 
8. Strawberries
9. Limes 
10. Organic Whole Milk

First, write down which columns you need and which dataframes have them.

Next, merge these into a single dataframe.

Then, use pandas functions from the previous lesson to get the counts of the top 10 most frequently ordered products.

In [5]:
##### YOUR CODE HERE #####
import pandas as pd
import numpy as np
df1 = pd.read_csv('aisles.csv')
df2 = pd.read_csv('departments.csv')
df3 = pd.read_csv('order_products__prior.csv')
df4 = pd.read_csv('order_products__train.csv')
df5 = pd.read_csv('orders.csv')
df6 = pd.read_csv('products.csv')

FileNotFoundError: ignored

In [0]:
df1.head()


In [0]:
df2.head()

In [0]:
df3.head()

In [0]:
df4.head()

In [0]:
df5.head()

In [0]:
df6.head()

In [0]:
# I need the following: product_id, reordered, product_name
# These are found in df6 and df3 (I assuem df4 is just a training b)

In [0]:
# I am trying to figure out the diff between 'order_products__prior.csv'and 
# 'order_products__train.csv'.  The explanations on 
# https://www.kaggle.com/c/instacart-market-basket-analysis/data isn't helpful. 
# In the real world, I'd follow up with the product owner to see if they can 
# explain the different to me.  But here, I am going to take the largest data 
# base. 
print(df4.shape)

In [0]:
print(df3.shape)

In [0]:
# I am going to use 'order_products__prior.csv because its shape is larger and contains more data. 
# I need to merge df3 and df6 based on product_id
# I am going to use an inner join. 
df7 = pd.merge(df3, df6, on='product_id', how='inner')
df7.head()

In [0]:
df8 = pd.pivot_table(df7, values='reordered', index='product_name', columns=None, aggfunc=np.sum)
df8.head(100)

In [0]:
df8.sort_values(by=['reordered'], ascending=False)


In [0]:
pd.set_option("display.max_rows",150)


In [0]:
df8.sort_values(by=['reordered'], ascending=False)



In [0]:
pd.get_option("display.max_rows")


In [0]:
# I am not sure why I can only see the first 5 and last 5 rows above. 
df8.sort_values(by=['reordered'], ascending=False).head(15)
# My order is different. Does that mean I've made a mistake?

## Reshape Data Section

- Replicate the lesson code. Remember, if you haven't followed along typing out what we did during lecture, do that now to get more repetition with the syntax.
- Take table 2 (the transpose of table 1) and practice changing it into Tidy format and back again. You should not use the transpose operation anywhere in this code:
 - Table 2 --> Tidy
 - Tidy --> Table 2
- Load seaborn's `flights` dataset by running the cell below. Then create a pivot table showing the number of passengers by month and year. Use year for the index and month for the columns. You've done it right if you get 112 passengers for January 1949 and 432 passengers for December 1960.

In [8]:
# Replication of the lesson
'Why did' + ' ' + 'the Patriots lose?'

'Why did the Patriots lose?'

In [0]:
df1 = pd.DataFrame({'a': [1,2,3,4], 'b':[4,5,6,7], 'c':[7,8,9,10]})
df2 = pd.DataFrame({'a': [6,7,8,7], 'b': [9,4,3,2], 'c': [1,6,2,9]})

In [10]:
df1.head()

,a,b,c
0,1,4,7
1,2,5,8
2,3,6,9
3,4,7,10


In [11]:
df2.head()

,a,b,c
0,6,9,1
1,7,4,6
2,8,3,2
3,7,2,9


In [12]:
concatenated_by_rows = pd.concat([df1,df2])
concatenated_by_rows.head(8)

,a,b,c
0,1,4,7
1,2,5,8
2,3,6,9
3,4,7,10
0,6,9,1
1,7,4,6
2,8,3,2
3,7,2,9


In [13]:
concatenated_by_columns = pd.concat([df1,df2], axis=1)
concatenated_by_columns.head()

,a,b,c,a,b,c
0,1,4,7,6,9,1
1,2,5,8,7,4,6
2,3,6,9,8,3,2
3,4,7,10,7,2,9


In [0]:
df3 = pd.DataFrame({'a': [4,3,2,1], 'b': [4,5,6,7], 'c': [7,8,9,10]})
df4 = pd.DataFrame({'a': [6,4,8,7,8], 'b': [9,4,3,2,1], 'd': [1,6,2,9,5]})

In [15]:
df3.head()

,a,b,c
0,4,4,7
1,3,5,8
2,2,6,9
3,1,7,10


In [16]:
df4.head()

,a,b,d
0,6,9,1
1,4,4,6
2,8,3,2
3,7,2,9
4,8,1,5


In [17]:
concatenated_by_rows = pd.concat([df3,df4], sort=True)
concatenated_by_rows.head(9)

,a,b,c,d
0,4,4,7.0,NaN
1,3,5,8.0,NaN
2,2,6,9.0,NaN
3,1,7,10.0,NaN
0,6,9,NaN,1.0
1,4,4,NaN,6.0
2,8,3,NaN,2.0
3,7,2,NaN,9.0
4,8,1,NaN,5.0


In [18]:
concatenated_by_columns = pd.concat([df3,df4], axis=1)
concatenated_by_columns.head()

,a,b,c,a,b,d
0,4.0,4.0,7.0,6,9,1
1,3.0,5.0,8.0,4,4,6
2,2.0,6.0,9.0,8,3,2
3,1.0,7.0,10.0,7,2,9
4,NaN,NaN,NaN,8,1,5


In [19]:
from IPython.display import display, Image
url = 'https://cdn-images-1.medium.com/max/1600/1*vYGFQCafJtGBBX5mbl0xyw.png'
example = Image(url=url, width=600)

display(example)

In [20]:
!wget https://s3.amazonaws.com/instacart-datasets/instacart_online_grocery_shopping_2017_05_01.tar.gz

--2020-01-23 02:54:03--  https://s3.amazonaws.com/instacart-datasets/instacart_online_grocery_shopping_2017_05_01.tar.gz
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.216.108.61
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.216.108.61|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 205548478 (196M) [application/x-gzip]
Saving to: ‘instacart_online_grocery_shopping_2017_05_01.tar.gz.1’

instacart_online_gr 100%[===================>] 196.03M  16.4MB/s    in 13s     

2020-01-23 02:54:17 (14.6 MB/s) - ‘instacart_online_grocery_shopping_2017_05_01.tar.gz.1’ saved [205548478/205548478]



In [21]:
!tar --gunzip --extract --verbose --file=instacart_online_grocery_shopping_2017_05_01.tar.gz

instacart_2017_05_01/
instacart_2017_05_01/._aisles.csv
instacart_2017_05_01/aisles.csv
instacart_2017_05_01/._departments.csv
instacart_2017_05_01/departments.csv
instacart_2017_05_01/._order_products__prior.csv
instacart_2017_05_01/order_products__prior.csv
instacart_2017_05_01/._order_products__train.csv
instacart_2017_05_01/order_products__train.csv
instacart_2017_05_01/._orders.csv
instacart_2017_05_01/orders.csv
instacart_2017_05_01/._products.csv
instacart_2017_05_01/products.csv


In [22]:
%cd instacart_2017_05_01

/content/instacart_2017_05_01/instacart_2017_05_01


In [23]:
!ls -1h *.csv

aisles.csv
departments.csv
order_products__prior.csv
order_products__train.csv
orders.csv
products.csv


In [24]:
aisles = pd.read_csv('aisles.csv')
print (aisles.shape)
aisles.head()

(134, 2)


,aisle_id,aisle
0,1,prepared soups salads
1,2,specialty cheeses
2,3,energy granola bars
3,4,instant foods
4,5,marinades meat preparation


In [25]:
departments = pd.read_csv('departments.csv')
print(departments.shape)
departments.head()

(21, 2)


,department_id,department
0,1,frozen
1,2,other
2,3,bakery
3,4,produce
4,5,alcohol


In [26]:
order_products__prior = pd.read_csv('order_products__prior.csv')
print(order_products__prior.shape)
order_products__prior.head()


(32434489, 4)


,order_id,product_id,add_to_cart_order,reordered
0,2,33120,1,1
1,2,28985,2,1
2,2,9327,3,0
3,2,45918,4,1
4,2,30035,5,0


In [27]:
order_products__train = pd.read_csv('order_products__train.csv')
print(order_products__train.shape)
order_products__train.head()

(1384617, 4)


,order_id,product_id,add_to_cart_order,reordered
0,1,49302,1,1
1,1,11109,2,1
2,1,10246,3,0
3,1,49683,4,0
4,1,43633,5,1


In [28]:
orders = pd.read_csv('orders.csv')
print(orders.shape)
orders.head()

(3421083, 7)


,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,2539329,1,prior,1,2,8,NaN
1,2398795,1,prior,2,3,7,15.0
2,473747,1,prior,3,3,12,21.0
3,2254736,1,prior,4,4,7,29.0
4,431534,1,prior,5,4,15,28.0


In [29]:
products = pd.read_csv('products.csv')
print(products.shape)
products.head()

(49688, 4)


,product_id,product_name,aisle_id,department_id
0,1,Chocolate Sandwich Cookies,61,19
1,2,All-Seasons Salt,104,13
2,3,Robust Golden Unsweetened Oolong Tea,94,7
3,4,Smart Ones Classic Favorites Mini Rigatoni Wit...,38,1
4,5,Green Chile Anytime Sauce,5,13


In [30]:
print(order_products__prior.shape)
print(order_products__train.shape)

(32434489, 4)
(1384617, 4)


In [31]:
order_products = pd.concat([order_products__prior, order_products__train])
print (order_products.shape)
order_products.head()

(33819106, 4)


,order_id,product_id,add_to_cart_order,reordered
0,2,33120,1,1
1,2,28985,2,1
2,2,9327,3,0
3,2,45918,4,1
4,2,30035,5,0


In [0]:
# Note to self: Concatenating dataframes means to stick two dataframes together either by rows or by columns. The default behavior of pd.concat() 
#is to take the rows of one dataframe and add them to the rows of another dataframe. If we pass the argument axis=1 then we will be adding the columns of one dataframe to the columns of another dataframe. 
# Concatenating dataframes is most useful when the columns are the same between two dataframes or when we have matching row indices between two dataframes.
# Whenever two dataframes hold different kinds of information about the same item but share a common feature/column we can use the linking feature as an index and merge two dataframes together based on the common column. To accomplish this we can use either the .merge() or .join() operations.



In [32]:
display (example)


In [33]:
condition = (orders['order_id'] <= 5)
orders_subset = orders[condition]
orders_subset.head()

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
1868044,1,112108,train,4,4,10,9.0
2593373,5,156122,prior,42,6,16,9.0
2958007,4,178520,prior,36,1,9,7.0
3355525,2,202279,prior,3,5,9,8.0
3417191,3,205970,prior,16,5,17,12.0


In [36]:
condition = ((orders['user_id'] == 1) & (orders['order_number'] <= 2))
orders_subset = orders[condition]
print(orders_subset.shape)
orders_subset.head()

(2, 7)


,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,2539329,1,prior,1,2,8,NaN
1,2398795,1,prior,2,3,7,15.0


In [37]:
orders_subset = orders[]

SyntaxError: ignored

In [39]:
order_products_subset = order_products[((order_products['order_id'] == 2539329) | (order_products['order_id'] == 2398795))]

print(order_products_subset.shape)
order_products_subset.head(11)
# Can someone tell me what | this fucntion is? 

(11, 4)


,order_id,product_id,add_to_cart_order,reordered
22742744,2398795,196,1,1
22742745,2398795,10258,2,0
22742746,2398795,12427,3,1
22742747,2398795,13176,4,0
22742748,2398795,26088,5,1
22742749,2398795,13032,6,0
24076664,2539329,196,1,0
24076665,2539329,14084,2,0
24076666,2539329,12427,3,0
24076667,2539329,26088,4,0


In [40]:
orders_and_products = pd.merge(orders_subset, order_products_subset, on='order_id', how='inner')
orders_and_products.head(11)

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered
0,2539329,1,prior,1,2,8,NaN,196,1,0
1,2539329,1,prior,1,2,8,NaN,14084,2,0
2,2539329,1,prior,1,2,8,NaN,12427,3,0
3,2539329,1,prior,1,2,8,NaN,26088,4,0
4,2539329,1,prior,1,2,8,NaN,26405,5,0
5,2398795,1,prior,2,3,7,15.0,196,1,1
6,2398795,1,prior,2,3,7,15.0,10258,2,0
7,2398795,1,prior,2,3,7,15.0,12427,3,1
8,2398795,1,prior,2,3,7,15.0,13176,4,0
9,2398795,1,prior,2,3,7,15.0,26088,5,1


In [41]:
display(example)
orders_and_products = orders_and_products.drop(['eval_set', 'reordered', 'days_since_prior_order'], axis='columns')
orders_and_products.head(11)

,order_id,user_id,order_number,order_dow,order_hour_of_day,product_id,add_to_cart_order
0,2539329,1,1,2,8,196,1
1,2539329,1,1,2,8,14084,2
2,2539329,1,1,2,8,12427,3
3,2539329,1,1,2,8,26088,4
4,2539329,1,1,2,8,26405,5
5,2398795,1,2,3,7,196,1
6,2398795,1,2,3,7,10258,2
7,2398795,1,2,3,7,12427,3
8,2398795,1,2,3,7,13176,4
9,2398795,1,2,3,7,26088,5


In [42]:
orders_and_products['product_id']

0       196
1     14084
2     12427
3     26088
4     26405
5       196
6     10258
7     12427
8     13176
9     26088
10    13032
Name: product_id, dtype: int64

In [44]:
orders_and_products['product_id'].isin([196, 26088])


0      True
1     False
2     False
3      True
4     False
5      True
6     False
7     False
8     False
9      True
10    False
Name: product_id, dtype: bool

In [45]:
condition = products['product_id'].isin(orders_and_products['product_id'])
products_subset = products[condition]
products_subset

,product_id,product_name,aisle_id,department_id
195,196,Soda,77,7
10257,10258,Pistachios,117,19
12426,12427,Original Beef Jerky,23,19
13031,13032,Cinnamon Toast Crunch,121,14
13175,13176,Bag of Organic Bananas,24,4
14083,14084,Organic Unsweetened Vanilla Almond Milk,91,16
26087,26088,Aged White Cheddar Popcorn,23,19
26404,26405,XL Pick-A-Size Paper Towel Rolls,54,17


In [46]:
final = pd.merge(orders_and_products, products_subset, on='product_id', how='inner')
final

,order_id,user_id,order_number,order_dow,order_hour_of_day,product_id,add_to_cart_order,product_name,aisle_id,department_id
0,2539329,1,1,2,8,196,1,Soda,77,7
1,2398795,1,2,3,7,196,1,Soda,77,7
2,2539329,1,1,2,8,14084,2,Organic Unsweetened Vanilla Almond Milk,91,16
3,2539329,1,1,2,8,12427,3,Original Beef Jerky,23,19
4,2398795,1,2,3,7,12427,3,Original Beef Jerky,23,19
5,2539329,1,1,2,8,26088,4,Aged White Cheddar Popcorn,23,19
6,2398795,1,2,3,7,26088,5,Aged White Cheddar Popcorn,23,19
7,2539329,1,1,2,8,26405,5,XL Pick-A-Size Paper Towel Rolls,54,17
8,2398795,1,2,3,7,10258,2,Pistachios,117,19
9,2398795,1,2,3,7,13176,4,Bag of Organic Bananas,24,4


In [0]:
final = final.drop(['aisle_id', 'department_id'], axis=1)


In [48]:
final

,order_id,user_id,order_number,order_dow,order_hour_of_day,product_id,add_to_cart_order,product_name
0,2539329,1,1,2,8,196,1,Soda
1,2398795,1,2,3,7,196,1,Soda
2,2539329,1,1,2,8,14084,2,Organic Unsweetened Vanilla Almond Milk
3,2539329,1,1,2,8,12427,3,Original Beef Jerky
4,2398795,1,2,3,7,12427,3,Original Beef Jerky
5,2539329,1,1,2,8,26088,4,Aged White Cheddar Popcorn
6,2398795,1,2,3,7,26088,5,Aged White Cheddar Popcorn
7,2539329,1,1,2,8,26405,5,XL Pick-A-Size Paper Towel Rolls
8,2398795,1,2,3,7,10258,2,Pistachios
9,2398795,1,2,3,7,13176,4,Bag of Organic Bananas


In [49]:
final = final.sort_values(by=['order_number', 'add_to_cart_order'])
final

,order_id,user_id,order_number,order_dow,order_hour_of_day,product_id,add_to_cart_order,product_name
0,2539329,1,1,2,8,196,1,Soda
2,2539329,1,1,2,8,14084,2,Organic Unsweetened Vanilla Almond Milk
3,2539329,1,1,2,8,12427,3,Original Beef Jerky
5,2539329,1,1,2,8,26088,4,Aged White Cheddar Popcorn
7,2539329,1,1,2,8,26405,5,XL Pick-A-Size Paper Towel Rolls
1,2398795,1,2,3,7,196,1,Soda
8,2398795,1,2,3,7,10258,2,Pistachios
4,2398795,1,2,3,7,12427,3,Original Beef Jerky
9,2398795,1,2,3,7,13176,4,Bag of Organic Bananas
6,2398795,1,2,3,7,26088,5,Aged White Cheddar Popcorn


In [0]:
final = final[['user_id', 'order_id', 'order_number', 'order_dow', 'order_hour_of_day', 'add_to_cart_order', 'product_id', 'product_name']]
final.columns = [column.replace('_', ' ') for column in final]


In [50]:
final

,order_id,user_id,order_number,order_dow,order_hour_of_day,product_id,add_to_cart_order,product_name
0,2539329,1,1,2,8,196,1,Soda
2,2539329,1,1,2,8,14084,2,Organic Unsweetened Vanilla Almond Milk
3,2539329,1,1,2,8,12427,3,Original Beef Jerky
5,2539329,1,1,2,8,26088,4,Aged White Cheddar Popcorn
7,2539329,1,1,2,8,26405,5,XL Pick-A-Size Paper Towel Rolls
1,2398795,1,2,3,7,196,1,Soda
8,2398795,1,2,3,7,10258,2,Pistachios
4,2398795,1,2,3,7,12427,3,Original Beef Jerky
9,2398795,1,2,3,7,13176,4,Bag of Organic Bananas
6,2398795,1,2,3,7,26088,5,Aged White Cheddar Popcorn


In [0]:
# Wow, this takes a long time if you really follow along and do key stroke by key stroke. Going to stop doing that. 




In [69]:
import pandas as pd
import numpy as np
import seaborn as sns

table1 = pd.DataFrame(
    [[np.nan, 2],
     [16,    11], 
     [3,      1]],
    index=['John Smith', 'Jane Doe', 'Mary Johnson'], 
    columns=['treatmenta', 'treatmentb'])

table1

,treatmenta,treatmentb
John Smith,NaN,2
Jane Doe,16.0,11
Mary Johnson,3.0,1


In [70]:
table2 = table1.T

table2


,John Smith,Jane Doe,Mary Johnson
treatmenta,NaN,16.0,3.0
treatmentb,2.0,11.0,1.0


In [71]:
# table1 -> tidy
table1=table1.reset_index()
table1

,index,treatmenta,treatmentb
0,John Smith,NaN,2
1,Jane Doe,16.0,11
2,Mary Johnson,3.0,1


In [72]:
tidy1 = table1.melt(id_vars='index', value_vars=['treatmenta', 'treatmentb'])
tidy1

,index,variable,value
0,John Smith,treatmenta,NaN
1,Jane Doe,treatmenta,16.0
2,Mary Johnson,treatmenta,3.0
3,John Smith,treatmentb,2.0
4,Jane Doe,treatmentb,11.0
5,Mary Johnson,treatmentb,1.0


In [73]:
tidy1 = tidy1.rename(columns={'index': 'name', 'variable': 'trt', 'value': 'result'})
tidy1

,name,trt,result
0,John Smith,treatmenta,NaN
1,Jane Doe,treatmenta,16.0
2,Mary Johnson,treatmenta,3.0
3,John Smith,treatmentb,2.0
4,Jane Doe,treatmentb,11.0
5,Mary Johnson,treatmentb,1.0


In [74]:
tidy1.trt = tidy1.trt.str.replace('treatment', '')
tidy1

,name,trt,result
0,John Smith,a,NaN
1,Jane Doe,a,16.0
2,Mary Johnson,a,3.0
3,John Smith,b,2.0
4,Jane Doe,b,11.0
5,Mary Johnson,b,1.0


In [78]:
# table2 -> tidy
table2

,John Smith,Jane Doe,Mary Johnson
treatmenta,NaN,16.0,3.0
treatmentb,2.0,11.0,1.0


In [79]:
table2 = table2.T
table2



,treatmenta,treatmentb
John Smith,NaN,2.0
Jane Doe,16.0,11.0
Mary Johnson,3.0,1.0


In [80]:
table2=table2.reset_index()
table2



,index,treatmenta,treatmentb
0,John Smith,NaN,2.0
1,Jane Doe,16.0,11.0
2,Mary Johnson,3.0,1.0


In [82]:
tidy2 = table2.melt(id_vars='index')
tidy2
#table2 = table2.pivot(index=None, columns='index')
 #table2=table2.reset_index()

#tidy2 = table2.melt(id_vars='index', value_vars=['treatmenta', 'treatmentb'])


,index,variable,value
0,John Smith,treatmenta,NaN
1,Jane Doe,treatmenta,16.0
2,Mary Johnson,treatmenta,3.0
3,John Smith,treatmentb,2.0
4,Jane Doe,treatmentb,11.0
5,Mary Johnson,treatmentb,1.0


In [83]:
tidy2 = tidy2.rename(columns={'index': 'name', 'variable': 'trt', 'value': 'result'})
tidy2

,name,trt,result
0,John Smith,treatmenta,NaN
1,Jane Doe,treatmenta,16.0
2,Mary Johnson,treatmenta,3.0
3,John Smith,treatmentb,2.0
4,Jane Doe,treatmentb,11.0
5,Mary Johnson,treatmentb,1.0


In [84]:
tidy2.trt = tidy2.trt.str.replace('treatment', '')
tidy2

,name,trt,result
0,John Smith,a,NaN
1,Jane Doe,a,16.0
2,Mary Johnson,a,3.0
3,John Smith,b,2.0
4,Jane Doe,b,11.0
5,Mary Johnson,b,1.0


In [86]:
# Flights Pivot Table
flights = sns.load_dataset('flights')
flights

,year,month,passengers
0,1949,January,112
1,1949,February,118
2,1949,March,132
3,1949,April,129
4,1949,May,121
...,...,...,...
139,1960,August,606
140,1960,September,508
141,1960,October,461
142,1960,November,390


In [92]:
flights2 = flights.pivot_table(index='month', columns='year', values='passengers')
flights2



year,1949,1950,1951,1952,1953,1954,1955,1956,1957,1958,1959,1960
month,,,,,,,,,,,,
January,112,115,145,171,196,204,242,284,315,340,360,417
February,118,126,150,180,196,188,233,277,301,318,342,391
March,132,141,178,193,236,235,267,317,356,362,406,419
April,129,135,163,181,235,227,269,313,348,348,396,461
May,121,125,172,183,229,234,270,318,355,363,420,472
June,135,149,178,218,243,264,315,374,422,435,472,535
July,148,170,199,230,264,302,364,413,465,491,548,622
August,148,170,199,242,272,293,347,405,467,505,559,606
September,136,158,184,209,237,259,312,355,404,404,463,508


## Join Data Stretch Challenge

The [Instacart blog post](https://tech.instacart.com/3-million-instacart-orders-open-sourced-d40d29ead6f2) has a visualization of "**Popular products** purchased earliest in the day (green) and latest in the day (red)." 

The post says,

> "We can also see the time of day that users purchase specific products.

> Healthier snacks and staples tend to be purchased earlier in the day, whereas ice cream (especially Half Baked and The Tonight Dough) are far more popular when customers are ordering in the evening.

> **In fact, of the top 25 latest ordered products, the first 24 are ice cream! The last one, of course, is a frozen pizza.**"

Your challenge is to reproduce the list of the top 25 latest ordered popular products.

We'll define "popular products" as products with more than 2,900 orders.



In [0]:
##### YOUR CODE HERE #####

## Reshape Data Stretch Challenge

_Try whatever sounds most interesting to you!_

- Replicate more of Instacart's visualization showing "Hour of Day Ordered" vs "Percent of Orders by Product"
- Replicate parts of the other visualization from [Instacart's blog post](https://tech.instacart.com/3-million-instacart-orders-open-sourced-d40d29ead6f2), showing "Number of Purchases" vs "Percent Reorder Purchases"
- Get the most recent order for each user in Instacart's dataset. This is a useful baseline when [predicting a user's next order](https://www.kaggle.com/c/instacart-market-basket-analysis)
- Replicate parts of the blog post linked at the top of this notebook: [Modern Pandas, Part 5: Tidy Data](https://tomaugspurger.github.io/modern-5-tidy.html)

In [0]:
##### YOUR CODE HERE #####